https://huggingface.co/CompendiumLabs/bge-large-en-v1.5-gguf

- https://pypi.org/project/llama-cpp-python/
- https://pypi.org/project/scikit-learn/
- https://pypi.org/project/beautifulsoup4/

requirements.txt ->
llama-cpp-python
scikit-learn 
scipy 
numpy

In [1]:
# !pip install -q llama-cpp-python
# !pip install -q scikit-learn scipy numpy
# !pip install -q beautifulsoup4

In [2]:
!ls

 bge-large-en-v1.5-f32.gguf
 bge-large-en-v1.5-q4_k_m.gguf
'embedding_model_llama_cpp_search_match_v1 (copy).ipynb'
 embedding_model_llama_cpp_search_match_v1.ipynb
 embedding_model_llama_cpp_search_match_v2.ipynb
 embedding_model_llama_cpp_search_match_v3.ipynb
 embedding_model_llama_cpp_search_match_v4.ipynb
 embedding_model_llama_cpp_search_match_v5.ipynb
 embedding_model_llama_cpp_search_match_v6.ipynb
 embedding_model_llama_cpp_test.ipynb
 env
 requirements.txt


In [3]:
model_path = "bge-large-en-v1.5-f32.gguf"

In [4]:
text_to_embed = "hello world"

In [5]:
from llama_cpp import Llama
model = Llama(model_path, embedding=True)


llama_model_loader: loaded meta data with 23 key-value pairs and 389 tensors from bge-large-en-v1.5-f32.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = bert
llama_model_loader: - kv   1:                               general.name str              = bge-large-en-v1.5
llama_model_loader: - kv   2:                           bert.block_count u32              = 24
llama_model_loader: - kv   3:                        bert.context_length u32              = 512
llama_model_loader: - kv   4:                      bert.embedding_length u32              = 1024
llama_model_loader: - kv   5:                   bert.feed_forward_length u32              = 4096
llama_model_loader: - kv   6:                  bert.attention.head_count u32              = 16
llama_model_loader: - kv   7:          bert.attention.layer_norm_epsilon f32   

In [6]:
embed = model.embed(text_to_embed)


llama_print_timings:        load time =     105.34 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =     100.97 ms /     4 tokens (   25.24 ms per token,    39.62 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =     104.68 ms /     5 tokens


In [7]:
## inspection to see if pipeline is working
# print(embed)

In [8]:
embedding1 = model.embed("flower")
embedding2 = model.embed("blossom")


llama_print_timings:        load time =     105.34 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =     174.63 ms /     3 tokens (   58.21 ms per token,    17.18 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =     176.83 ms /     4 tokens

llama_print_timings:        load time =     105.34 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =      93.18 ms /     3 tokens (   31.06 ms per token,    32.19 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =      96.44 ms /     4 tokens


In [9]:
#############################################
# get_vector function to modularize pipeline
#############################################
def get_vector(text_to_vectorize):
    return model.embed(text_to_vectorize)

In [10]:
# step 1: embed the search-phrase
# step 2: embed each text
# step 3: get scores
# step 4: evaluates if score is succss or fail
# step 5: if success: do stuff with text, else: move on

In [11]:


from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_distance(embedding1, embedding2, boolean=False, threshold=0.5):
    """
    Cosine Similarity: This is a common method for measuring the similarity
    between two vectors. It measures the cosine of the angle between
    two vectors and the result is a value between -1 and 1.
    A value of 1 means the vectors are identical,
    0 means they are orthogonal (or completely dissimilar),
    and -1 means they are diametrically opposed.
    """
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = cosine_similarity([embedding1], [embedding2])

    similarity = similarity[0][0]
    
    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True


from scipy.spatial.distance import euclidean

def euclidean_distance(embedding1, embedding2, boolean=False, threshold=0.5):
    """
    Euclidean Distance: This is another common method for measuring
     the similarity between two vectors.
     It calculates the straight-line distance between two points in a space.
     The smaller the distance, the more similar the vectors.
    """
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + euclidean(embedding1, embedding2))
    
    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True



import numpy as np

def dot_product(embedding1, embedding2, boolean=False, threshold=0.5):
    """
    Dot Product: This is a simple method that calculates
    the sum of the products of the corresponding entries of the
    two sequences of numbers. If the vectors are normalized,
    the dot product is equal to the cosine similarity.
    """
    # Assuming embedding1 and embedding2 are your embeddings
    dot_product = np.dot(embedding1, embedding2)
    normalized_dot_product = dot_product / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

    similarity = normalized_dot_product
    
    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True


from scipy.spatial.distance import cityblock

def manhattan_distance(embedding1, embedding2, boolean=False, threshold=0.5):
    """
    Manhattan Distance: This is a measure of the distance between
    two vectors in a grid-based system.
    It calculates the sum of the absolute differences of their coordinates.
    """
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + cityblock(embedding1, embedding2))


    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True



from scipy.stats import pearsonr

def pearson_correlation(embedding1, embedding2, boolean=False, threshold=0.5):
    """
    Pearson Correlation: This is a measure of the linear correlation
    between two vectors. It ranges from -1 (perfectly negatively correlated)
     to 1 (perfectly positively correlated).
    """
    
    # Assuming embedding1 and embedding2 are your embeddings
    similarity, _ = pearsonr(embedding1, embedding2)

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True



from scipy.stats import spearmanr

def spearmans_rank_correlation(embedding1, embedding2, boolean=False, threshold=0.5):
    """
    Spearman's Rank Correlation: This is a non-parametric
     measure of the monotonicity of the relationship between
     two datasets. Unlike the Pearson correlation, the Spearman
      correlation does not assume that the relationship between
       the two variables is linear.
    """
    
    # Assuming embedding1 and embedding2 are your embeddings
    similarity, _ = spearmanr(embedding1, embedding2)


    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True


from scipy.stats import kendalltau
def kendalls_rank_correlation(embedding1, embedding2, boolean=False, threshold=0.5):
    
    """
    Kendall's Rank Correlation: This is another non-parametric
    measure of the ordinal association between two variables.
    It is a measure of the correspondence between two rankings.
    """
    
    # Assuming embedding1 and embedding2 are your embeddings
    similarity, _ = kendalltau(embedding1, embedding2)

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True



from scipy.spatial.distance import minkowski


def minkowski_distance(embedding1, embedding2, boolean=False, threshold=0.5):
    """
    Minkowski Distance: This is a generalization of
    both the Euclidean distance and the Manhattan distance.
    It is defined as the p-th root of the sum of the p-th powers
    of the differences of the coordinates.
    When p=1, this is the Manhattan distance,
    and when p=2, this is the Euclidean distance.
    """
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + minkowski(embedding1, embedding2, p=2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True



from scipy.spatial.distance import chebyshev
def chebyshev_distance(embedding1, embedding2, boolean=False, threshold=0.5):
    """
    Chebyshev Distance: This is a measure of the distance between
    two vectors in a vector space.
    It is the maximum of the absolute differences of their coordinates.
    """
    
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + chebyshev(embedding1, embedding2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True



import numpy as np
from scipy.spatial.distance import mahalanobis
from numpy.linalg import inv

def mahalanobis_distance(embedding1, embedding2, boolean=False, threshold=0.415):
    """Mahalanobis Distance: This is a measure of the distance between 
    a point P and a distribution D, introduced by P. C. Mahalanobis in 1936.
    It is a multivariate generalization of the Euclidean distance.
    It is based on correlations between dimensions of the data, 
    and thus takes into account the structure of the data.
    """

    # Assuming embedding1 and embedding2 are your vectors
    data = np.array([embedding1, embedding2])

    # Calculate the covariance matrix with a small regularization term
    cov = np.cov(data, rowvar=False) + np.eye(data.shape[1])# * 1e-6

    # Calculate the Mahalanobis distance
    distance = mahalanobis(embedding1, embedding2, inv(cov))

    # Calculate the similarity score
    similarity = 1 / (1 + distance)

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True




from scipy.spatial.distance import braycurtis
def bray_curtis_distance_dissimilarity(embedding1, embedding2, boolean=False, threshold=0.5):
    """Bray-Curtis Distance: This is a measure of dissimilarity
    between two vectors. It is used in ecology to compare species
    composition in different samples. It is defined as the sum of
    the absolute differences between the vectors, divided by the sum of their sums.
    """
    
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + braycurtis(embedding1, embedding2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True



from scipy.spatial.distance import canberra
def canberra_distance(embedding1, embedding2, boolean=False, threshold=0.5):
    """
    dissimilarity
    Canberra Distance: This is a measure of the dissimilarity
    between two vectors. It is defined as the sum of the absolute
    differences between the vectors, divided by the sum of their absolute values.
    """
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + canberra(embedding1, embedding2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True




from scipy.stats import pearsonr
def correlation_distance_dissimilarity_measure(embedding1, embedding2, boolean=False, threshold=0.5):
    """
    dissimilarity
    Correlation Distance: This is a measure of the dissimilarity
    between two vectors. It is defined as 1 - the absolute value of
    the Pearson correlation coefficient between the vectors.
    """
    # Assuming embedding1 and embedding2 are your embeddings
    correlation, _ = pearsonr(embedding1, embedding2)
    similarity = 1 / (1 + (1 - abs(correlation)))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True




from scipy.spatial.distance import sqeuclidean
def squared_euclidean_distance_dissimilarity_measure(embedding1, embedding2, boolean=False, threshold=0.5):
    """
    dissimilarity
    Squared Euclidean Distance: This is a measure of the dissimilarity
    between two vectors. It is defined as the sum of the squared differences
    between the vectors. It is similar to the Euclidean distance,
    but it does not take the square root, which can make it faster to compute.
    """
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + sqeuclidean(embedding1, embedding2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True


from scipy.spatial.distance import hamming
def hamming_distance_dissimilarity_measure(embedding1, embedding2):
    """
    Hamming Distance: This is a measure of the minimum number
    of substitutions required to change one vector into the other.
    It is used in information theory to measure the difference between
    two binary vectors.
    """
    # Assuming embedding1 and embedding2 are your binary vectors
    similarity = 1 / (1 + hamming(embedding1, embedding2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True



# """
# not for vectors
# dissimilarity
# Jensen-Shannon Distance: This is a measure of the dissimilarity
# between two probability distributions. It is defined as the square root
# of the Jensen-Shannon divergence. It is a symmetric and smooth measure
# of dissimilarity that is always greater than or equal to 0.
# """
# from scipy.spatial.distance import jensenshannon
# import numpy as np
# def jensen_shannon_distance_dissimilarity_measure(embedding1, embedding2):
#     # Assuming embedding1 and embedding2 are your probability distributions
#     # Compute the average of the two distributions
#     average = 0.5 * np.add(embedding1, embedding2)
#     # Compute the Jensen-Shannon divergence
#     jsd = 0.5 * jensenshannon(embedding1, average) + 0.5 * jensenshannon(embedding2, average)
#     # Compute the Jensen-Shannon distance
#     dissimilarity = np.sqrt(jsd)
#     return dissimilarity



# """
# not for vectors
# dissimilarity
# Kullback-Leibler Divergence:
# This is a measure of the dissimilarity between two probability distributions.
#  It is not symmetric, meaning that the divergence from
#  distribution P to distribution Q is not
#  necessarily the same as the divergence from Q to P.
# """
# from scipy.special import kl_div
# def kullback_leibler_distance_dissimilarity_measure(embedding1, embedding2):
#     # Assuming embedding1 and embedding2 are your probability distributions
#     # Compute the Kullback-Leibler divergence
#     kld = np.sum(kl_div(embedding1, embedding2))
#     return kld



from scipy.stats import wasserstein_distance
def total_variation_distance_dissimilarity_measure(embedding1, embedding2, boolean=False, threshold=0.5):
    """
    dissimilarity
    Total Variation Distance: This is a measure of the dissimilarity
    between two probability distributions.
    It is defined as half the sum of the absolute differences
    between the corresponding probabilities in the two distributions.
    """
    # Assuming embedding1 and embedding2 are your probability distributions
    similarity = 1 / (1 + wasserstein_distance(embedding1, embedding2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if simialirity < threshold:
            return False

        else:
            return True








In [12]:

# # step 1: embed the search-phrase
# # step 2: embed each text
# # step 3: get scores
# # step 4: evaluates if score is succss or fail
# # step 5: if success: do stuff with text, else: move on

# # arxiv inspector


# import requests
# from bs4 import BeautifulSoup
# import json

# ##########################################
# # Make comparison phrase and vectorize it
# ##########################################
# comparison_phrase = "Neural Networks"

# embedding1 = get_vector(comparison_phrase)


# # Make a request to the website
# r = requests.get('https://arxiv.org/list/cs/new')
# soup = BeautifulSoup(r.text, 'html.parser')

# # Find all the articles
# articles = soup.find_all('div', class_='meta')

# # List to hold all article data
# article_data = []

# for article in articles:
#     # Extract the title
#     title = article.find('div', class_='list-title mathjax').text.strip().replace('Title:', '')

#     # Extract the abstract
#     abstract = article.find('p', class_='mathjax').text.strip()

#     # Extract the link
#     link_element = article.find('a', title='Abstract')
#     if link_element:
#         link = 'https://arxiv.org' + link_element['href']
#     else:
#         link = ''


#     extracted_article_string = title + " " + abstract


#     ##################################
#     # Do embedding search here:
#     ##################################

#     embedding2 = get_vector(extracted_article_string)


#     ##################################
#     # Do basic embedding search here:
#     ##################################

#     # List of functions
#     list_of_comparison_functions = [
#         cosine_similarity_distance,
#         euclidean_distance,
#         dot_product,
#         manhattan_distance,
#         pearson_correlation,
#         spearmans_rank_correlation,
#         kendalls_rank_correlation,
#         minkowski_distance,
#         chebyshev_distance,
#         mahalanobis_distance,
#         bray_curtis_distance_dissimilarity,
#         canberra_distance,
#         correlation_distance_dissimilarity_measure,
#         squared_euclidean_distance_dissimilarity_measure,
#         hamming_distance_dissimilarity_measure,
#         jensen_shannon_distance_dissimilarity_measure,
#         kullback_leibler_distance_dissimilarity_measure,
#         total_variation_distance_dissimilarity_measure,
#         ]

#     # Arguments to pass to the functions
#     arguments = (embedding1, embedding2)

#     print(f"\n\n For {comparison_phrase} vs. {extracted_article_string[:200]}")
    
#     # Iterate through the functions and call each one with the arguments
#     for this_function in list_of_comparison_functions:
#         raw_score = this_function(*arguments)

#         print(raw_score)

#     input("PointBreak")


In [13]:

# step 1: embed the search-phrase
# step 2: embed each text
# step 3: get scores
# step 4: evaluates if score is succss or fail
# step 5: if success: do stuff with text, else: move on

# arxiv inspector


import requests
from bs4 import BeautifulSoup
import json

##########################################
# Make comparison phrase and vectorize it
##########################################
comparison_phrase = "I will not be coming to dinner."

embedding1 = get_vector(comparison_phrase)



articles = [
    "I will not be coming to dinner.",
    "I should not attend supper", 
    "When you dine tonight I will not be there",

    "For lunch people go to eat food.",
    "I will be eating eggs for breakfast", 
    
    "dogs dance",
    "rocks",
    "velocity",
]

# List to hold all article data
article_data = []

for article in articles:

    extracted_article_string = article


    ##################################
    # Do embedding search here:
    ##################################

    embedding2 = get_vector(extracted_article_string)


    ##################################
    # Do basic embedding search here:
    ##################################

    # List of functions
    list_of_comparison_functions = [
        mahalanobis_distance,
        canberra_distance,
        euclidean_distance,
        
        manhattan_distance,
        minkowski_distance,

        
        squared_euclidean_distance_dissimilarity_measure,


        chebyshev_distance,
        kendalls_rank_correlation,
        bray_curtis_distance_dissimilarity,
        cosine_similarity_distance,
        dot_product,



        spearmans_rank_correlation,






        
        pearson_correlation,

        # hamming_distance_dissimilarity_measure,

        correlation_distance_dissimilarity_measure,

        total_variation_distance_dissimilarity_measure,
        # jensen_shannon_distance_dissimilarity_measure,
        # kullback_leibler_distance_dissimilarity_measure,
        ]

    # Arguments to pass to the functions
    arguments = (embedding1, embedding2)

    print(f"For {comparison_phrase} vs. {extracted_article_string[:200]}")
    
    # Iterate through the functions and call each one with the arguments
    for this_function in list_of_comparison_functions:
        raw_score = this_function(*arguments)

        print(raw_score)

    # input("PointBreak")



llama_print_timings:        load time =     105.34 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =      59.90 ms /    10 tokens (    5.99 ms per token,   166.96 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =      61.12 ms /    11 tokens

llama_print_timings:        load time =     105.34 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =      93.54 ms /    10 tokens (    9.35 ms per token,   106.91 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =      95.70 ms /    11 tokens


For I will not be coming to dinner. vs. I will not be coming to dinner.
1.0
1.0


NameError: name 'normalized_dot_product' is not defined

In [ ]:

# step 1: embed the search-phrase
# step 2: embed each text
# step 3: get scores
# step 4: evaluates if score is succss or fail
# step 5: if success: do stuff with text, else: move on

# arxiv inspector


import requests
from bs4 import BeautifulSoup
import json

##########################################
# Make comparison phrase and vectorize it
##########################################
comparison_phrase = "computer vision"

embedding1 = get_vector(comparison_phrase)



articles = [
    """
    Principles of Designing Robust Remote Face Anti-Spoofing Systems Xiang Xu, Tianchen Zhao, Zheng Zhang, Zhihua Li, Jon Wu, Alessandro Achille, Mani Srivastava Comments: Under review Subjects: Computer Vision and Pattern Recognition (cs.CV); Cryptography and Security (cs.CR)
Protecting digital identities of human face from various attack vectors is paramount, and face anti-spoofing plays a crucial role in this endeavor. Current approaches primarily focus on detecting spoofing attempts within individual frames to detect presentation attacks. However, the emergence of hyper-realistic generative models capable of real-time operation has heightened the risk of digitally generated attacks. In light of these evolving threats, this paper aims to address two key aspects. First, it sheds light on the vulnerabilities of state-of-the-art face anti-spoofing methods against digital attacks. Second, it presents a comprehensive taxonomy of common threats encountered in face anti-spoofing systems. Through a series of experiments, we demonstrate the limitations of current face anti-spoofing detection techniques and their failure to generalize to novel digital attack scenarios. Notably, the existing models struggle with digital injection attacks including adversarial noise, realistic deepfake attacks, and digital replay attacks. To aid in the design and implementation of robust face anti-spoofing systems resilient to these emerging vulnerabilities, the paper proposes key design principles from model accuracy and robustness to pipeline robustness and even platform robustness. Especially, we suggest to implement the proactive face anti-spoofing system using active sensors to significant reduce the risks for unseen attack vectors and improve the user experience.
    """,
    """
    Principles of Designing Robust Remote Face Anti-Spoofing Systems Xiang Xu, Tianchen Zhao, Zheng Zhang, Zhihua Li, Jon Wu, Alessandro Achille, Mani Srivastava Comments: Under review Subjects: Computer Vision and Pattern Recognition (cs.CV); Cryptography and Security (cs.CR)

Protecting digital identities of human face from various attack vectors is paramount, and face anti-spoofing plays a crucial role in this endeavor. Current approaches primarily focus on detecting spoofing attempts within individual frames to detect presentation attacks. However, the emergence of hyper-realistic generative models capable of real-time operation has heightened the risk of digitally generated attacks. In light of these evolving threats, this paper aims to address two key aspects. First, it sheds light on the vulnerabilities of state-of-the-art face anti-spoofing methods against digital attacks. Second, it presents a comprehensive taxonomy of common threats encountered in face anti-spoofing systems. Through a series of experiments, we demonstrate the limitations of current face anti-spoofing detection techniques and their failure to generalize to novel digital attack scenarios. Notably, the existing models struggle with digital injection attacks including adversarial noise, realistic deepfake attacks, and digital replay attacks. To aid in the design and implementation of robust face anti-spoofing systems resilient to these emerging vulnerabilities, the paper proposes key design principles from model accuracy and robustness to pipeline robustness and even platform robustness. Especially, we suggest to implement the proactive face anti-spoofing system using active sensors to significant reduce the risks for unseen attack vectors and improve the user experience.
    """,
    """
    A Geometric View of Data Complexity: Efficient Local Intrinsic Dimension Estimation with Diffusion Models
Hamidreza Kamkari, Brendan Leigh Ross, Rasa Hosseinzadeh, Jesse C. Cresswell, Gabriel Loaiza-Ganem
Comments: 10 pages
Subjects: Machine Learning (cs.LG); Artificial Intelligence (cs.AI); Machine Learning (stat.ML)

High-dimensional data commonly lies on low-dimensional submanifolds, and estimating the local intrinsic dimension (LID) of a datum -- i.e. the dimension of the submanifold it belongs to -- is a longstanding problem. LID can be understood as the number of local factors of variation: the more factors of variation a datum has, the more complex it tends to be. Estimating this quantity has proven useful in contexts ranging from generalization in neural networks to detection of out-of-distribution data, adversarial examples, and AI-generated text. The recent successes of deep generative models present an opportunity to leverage them for LID estimation, but current methods based on generative models produce inaccurate estimates, require more than a single pre-trained model, are computationally intensive, or do not exploit the best available deep generative models, i.e. diffusion models (DMs). In this work, we show that the Fokker-Planck equation associated with a DM can provide a LID estimator which addresses all the aforementioned deficiencies. Our estimator, called FLIPD, is compatible with all popular DMs, and outperforms existing baselines on LID estimation benchmarks. We also apply FLIPD on natural images where the true LID is unknown. Compared to competing estimators, FLIPD exhibits a higher correlation with non-LID measures of complexity, better matches a qualitative assessment of complexity, and is the only estimator to remain tractable with high-resolution images at the scale of Stable Diffusion.
    """,

    "For lunch people go to eat food.",
    "I will be eating eggs for breakfast", 
    
    "dogs dance",
    "rocks",
    "velocity",
]

# List to hold all article data
article_data = []

for article in articles:

    extracted_article_string = article


    ##################################
    # Do embedding search here:
    ##################################

    embedding2 = get_vector(extracted_article_string)


    ##################################
    # Do basic embedding search here:
    ##################################

    # List of functions
    list_of_comparison_functions = [

        mahalanobis_distance,
        
        canberra_distance,
        
        euclidean_distance,
        
        manhattan_distance,
        
        minkowski_distance,

    
        squared_euclidean_distance_dissimilarity_measure,


        chebyshev_distance,
        
        kendalls_rank_correlation,
        
        bray_curtis_distance_dissimilarity,
        
        cosine_similarity_distance,
        
        dot_product,

        spearmans_rank_correlation,

        pearson_correlation,

        correlation_distance_dissimilarity_measure,

        total_variation_distance_dissimilarity_measure,

        ]

    # Arguments to pass to the functions
    arguments = (embedding1, embedding2)

    print(f"For {comparison_phrase} vs. {extracted_article_string[:100]}")
    
    # Iterate through the functions and call each one with the arguments
    for this_function in list_of_comparison_functions:
        raw_score = this_function(*arguments)

        print(raw_score)

    # input("PointBreak")
